## Problem Setup

### Poisson equation (2D)

$$-\Delta u =f(x,y)$$

 

### Problem

$$-\Delta u=2\pi^2sin(\pi x)sin(\pi y)$$

### Boundary Conditions:

$$u(-1,y)=0,\  u(1,y) =0, \ u(x,-1)=0,\  u(x,1) =0$$

### Exact solution:

$$u(x)=sin(\pi x)sin(\pi y)$$


So the residual will be:

$$0=\frac{\partial^2u}{\partial x^2}+\frac{\partial^2u}{\partial y^2}+2\pi^2sin(\pi x)sin(\pi y)$$

Note: Remeber that our neural network $NN(x,y)\approx u(x,y)$ so:

$$\frac{\partial^2NN}{\partial x^2}\approx\frac{\partial^2u}{\partial x^2}$$
$$\frac{\partial^2NN}{\partial y^2}\approx\frac{\partial^2u}{\partial y^2}$$

$$f=\frac{\partial^2NN}{\partial x^2}+\frac{\partial^2NN}{\partial y^2}+2\pi^2sin(\pi x)sin(\pi y)\rightarrow 0$$


**Initial Conditions (Dirichlet BC)**
$$u(-1,y)=0,\ u(1,y)=0$$

$$u(x,-1)=0,\ u(x,1)=0$$

Set two functions to describe our boundary conditions:

$$f_{BC_v}(x)=1-|x|,\  x = \pm  1 \ y \in (-1,1)$$
$$f_{BC_h}(x)=1-|y|,\  y = \pm 1 \ x \in (-1,1)$$

In [ ]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
#from sklearn.model_selection import train_test_split

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 


### Parameters setting

In [ ]:
steps=5000
lr=1e-3
layers = np.array([2,50,50,20,50,50,1]) #5 hidden layers
# To generate new data:
x_l=-1
x_r=1
y_l=-1
y_r=1
total_points=800
#Nu: Number of training points (2 as we onlt have 2 boundaries), # Nf: Number of collocation points (Evaluate PDE)
Nu=1000
Nf=10000

### Functions

In [ ]:
def f_BCv(x):
    return 1-torch.abs(x)

def f_BCh(y):
    return 1-torch.abs(y)

def f_real(x,y):
    return torch.sin(np.pi*x)*torch.sin(np.pi*y)

def PDE(x,y):
    return -2*(np.pi**2)*torch.sin(np.pi*x)*torch.sin(np.pi*y)

def plot3D(X,Y,f,title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X.numpy(), Y.numpy(), f.numpy(), cmap='jet', edgecolor='none')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('f(X, Y)')
    ax.set_title(title)
    plt.show()
    
    
def plot_heat(x_l,x_r,y_l,y_r,f,title_colorbar):
    plt.imshow(f.numpy(), cmap='jet', origin='lower', extent=[x_l, x_r, y_l, y_r])
    plt.colorbar(label='f(x, y)')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title_colorbar)

    plt.show()
    

### Neural Network

In [ ]:
class PINN(nn.Module):
    ##Neural Network
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
        'activation function'
        self.activation = nn.Tanh()
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.iter = 0
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):

            
            
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
    'foward pass'
    def forward(self,X):
        if torch.is_tensor(X) != True:         
            X = torch.from_numpy(X)                
        a = X.float()
        for i in range(len(layers)-2):  
            z = self.linears[i](a)              
            a = self.activation(z)    
        a = self.linears[-1](a)
        return a
    'Loss Functions'
    #Loss BC
    def lossBC(self,X_BC,f_BC):
        loss_BC=self.loss_function(self.forward(X_BC),f_BC)
        return loss_BC
    #Loss PDE
    def lossPDE(self,X_PDE):
        g=X_PDE.clone()
        g.requires_grad=True #Enable differentiation
        f=self.forward(g)
        f_X=autograd.grad(f,g,grad_outputs=torch.ones_like(f),create_graph=True)[0] #first derivative    
        grad_x =   f_X[:,0]
        grad_y = f_X[:,1]
        f_XX = autograd.grad(grad_x, g, grad_outputs=torch.ones_like(grad_x), create_graph=True)[0][:, 0]
        f_YY = autograd.grad(grad_y, g, grad_outputs=torch.ones_like(grad_y), create_graph=True)[0][:, 1]
        
        solu = 2*np.pi ** 2 *torch.sin(np.pi * g[:, 0:1])*torch.sin(np.pi * g[:, 1:])
        delta_f=f_XX.unsqueeze(1) + f_YY.unsqueeze(1) + solu
        
         
        
        
        return self.loss_function(delta_f,f_hat)
#     + 2*np.pi ** 2 *torch.sin(np.pi * g[:, 0:1])*torch.sin(np.pi * g[:, 1:])
        
    def loss(self,X_BC,f_BC,X_PDE):
        loss_bc=self.lossBC(X_BC,f_BC)
        loss_pde=self.lossPDE(X_PDE)
        return loss_bc+loss_pde
#X_train_Nu,v_train_Nu,X_train_Nf

### Data generation

In [ ]:
# Plot the function as a 3D heatmap
x = torch.linspace(x_l,x_r,total_points).view(-1,1) #prepare to NN
y = torch.linspace(y_l,y_r,total_points).view(-1,1) #prepare to NN

X, Y = torch.meshgrid(x.view(-1,), y.view(-1,))
f = f_real(X,Y)
 

In [ ]:
# Transform the mesh into a 2-column vector
x_test=torch.hstack((X.transpose(1,0).flatten()[:,None],Y.transpose(1,0).flatten()[:,None]))

v_test=f.transpose(1,0).flatten()[:,None] # Colum major Flatten (so we transpose it)
 
# Domain bounds
lb=x_test[0] #first value
ub=x_test[-1] #last value 

### Training Data

In [ ]:
#Boundary Conditions
#Left Edge 
left_X=torch.hstack((X[0,:][:,None],Y[0,:][:,None])) # First column # The [:,None] is to give it the right dimension
vleft_boundary = f_BCv(left_X[:,0]).unsqueeze(1)



#right Edge
right_X=torch.hstack((X[-1,:][:,None],Y[-1,:][:,None]))
vright_boundary = f_BCv(right_X[:,0]).unsqueeze(1)
 

#Bottom Edge
bottom_X=torch.hstack((X[:,0][:,None],Y[:,0][:,None])) # First row # The [:,None] is to give it the right dimension
vbottom_boundary = f_BCh(bottom_X[:,-1]) .unsqueeze(1)
 

#Top Edge
top_X=torch.hstack((X[:,0][:,None],Y[:,-1][:,None])) # First row # The [:,None] is to give it the right dimension
vtop_boundary = f_BCh(top_X[:,-1]).unsqueeze(1)
 
# #Get all the training data into the same dataset
X_train=torch.vstack([left_X,right_X,bottom_X,top_X])
v_train=torch.vstack([vleft_boundary,vright_boundary,vbottom_boundary,vtop_boundary])
#Choose(Nu) points of our available training data:
idx=np.random.choice(X_train.shape[0],Nu,replace=False)
X_train_Nu=X_train[idx,:]
v_train_Nu=v_train[idx,:]
  
# Collocation Points (Evaluate our PDe)
#Choose(Nf) points(Latin hypercube)
X_train_Nf=lb+(ub-lb)*lhs(2,Nf) # 2 as the inputs are x and y
X_train_Nf=torch.vstack((X_train_Nf,X_train_Nu)) #Add the training poinst to the collocation points
 


### Model training

In [ ]:
torch.manual_seed(123)
#Store tensors to GPU
X_train_Nu=X_train_Nu.float().to(device)#Training Points (BC)
v_train_Nu=v_train_Nu.float().to(device)#Training Points (BC)
X_train_Nf=X_train_Nf.float().to(device)#Collocation Points
f_hat = torch.zeros(X_train_Nf.shape[0],1).to(device)#to minimize function
print(f_hat.shape)

X_test=x_test.float().to(device) # the input dataset (complete)
V_test=v_test.float().to(device) # the real solution 


model = PINN(layers)
print(model)
model.to(device)
params = list(model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr=lr, amsgrad=False)
start_time = time.time()

In [ ]:
model.lossBC(X_train_Nu,v_train_Nu)

In [ ]:
%%time
for i in range(steps):
     
    loss = model.loss(X_train_Nu,v_train_Nu,X_train_Nf)# use mean squared error
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%200==0:
        print(loss)

In [ ]:
y_predict = model(x_test).detach()
 
reshaped_f = torch.reshape(y_predict, X.shape).transpose(1,0)

title = 'Plot of f(x,y) = sin(pi*x)*sin(pi*y)'
plot3D(X,Y,f,title)
title_colorbar = 'Color plot of f(x,y) = sin(pi*x)*sin(pi*y)'
plot_heat(x_l,x_r,y_l,y_r,f,title_colorbar)

title = 'Plot of predicted solution'
plot3D(X,Y,reshaped_f,title)
title_colorbar = 'Color plot of predicted solution'
plot_heat(x_l,x_r,y_l,y_r,reshaped_f,title_colorbar)

title_colorbar = 'Color plot of absolute error' 
plot_heat(x_l,x_r,y_l,y_r,reshaped_f-f,title_colorbar) 

### Data-driven discovery of partial differential equations

#### Continuous time models

In this section, the authors define the neural network that will be used to solve the inverse problems of partial differential equations which have parameters in the PDE:
$$u_t + \mathcal{N}(u,\lambda) = 0$$
Similar to the above Data-driven solutions of partial differential equations, we can define the neural network as
$$f(x,t) := u_t + \mathcal{N}(u,\lambda)$$
The parameters of the differential operator $\lambda$ turn into parameters of the physics-informed neural network.To illustrate the application of Physics-Informed Neural Networks (PINN) for solving partial differential equations, we provide the code implementation for the Poisson equation below:

## Problem Setup

### Burgers equation

$$\frac{\partial u}{\partial t}  + \lambda_1 u\frac{\partial u}{\partial x} - \lambda_2 \frac{\partial^2 u}{\partial x^2} =0$$
By using training data selected from the initial, boundary, and inner domain data, we can determine the parameters $\lambda_1$ and $\lambda_2$ in the Burgers equation. It is important to emphasize that our objective in this problem is to solve for the parameters in the Burgers equation, rather than obtaining a numerical solution of the equation itself. The training data can be chosen from the inner domain, and there is no need to identify specific initial and boundary training data or collocation points. To tackle this problem, we can treat the parameters of the Burgers equation as the parameters of the neural network, denoted as $f(\theta,x,t) := \frac{\partial u_{NN}(\theta,x,t)}{\partial t} + \lambda_1 u_{NN}(\theta,x,t) \frac{\partial u_{NN}(\theta,x,t)}{\partial x} - \lambda_2 \frac{\partial^2 u_{NN}(\theta,x,t)}{\partial x^2}$, where $\theta$ is the weight of the neural network. The loss function is the same as above:
$$MSE = MSE_u + MSE_f$$
where
$$MSE_u = \frac{1}{N_u}\sum_{i=1}^{N_u}|u(t^{i}_u,x_u^i)-u^{i}|^2$$
and 
$$MSE_f = \frac{1}{N_f}\sum_{i=1}^{N_f}|f(t^{i}_f,x_f^i)|^2$$

## Libraries

In [5]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from collections import OrderedDict

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling

import scipy.io
from scipy.interpolate import griddata
import sys
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

#Set default dtype to float32
torch.set_default_dtype(torch.float)

np.random.seed(1235)



# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

cuda


## Data Preparation

In [6]:
nu = 0.01/np.pi
N_u = 2000
#layers = [2, 20, 20, 20 ,1]
layers = [2, 50, 50, 50,  1]

data = scipy.io.loadmat('data/burgers_shock.mat')
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]     

# k_true =2#波数（单频）
# a1=1
# a2=4
# lin_num = 51
# t = np.linspace(3,5,lin_num)[:,None] #Y采样点
# x = np.linspace(3,5,lin_num)[:,None] #X采样点
# X, T = np.meshgrid(x,t)
# Exact = np.sin(k_true*X) * np.sin(k_true*T) #网格化坐标上的真解
# X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
# u_star = (0.5*np.sin(k_true*X) * np.sin(k_true* T)).flatten()[:,None]#网格化坐标上的真解      


# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0) 

## Physics-informed Neural Networks

In [7]:
# the deep neural network
class Solu_NN(torch.nn.Module):
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
    
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        self.iter = 0
        
    
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            
            # weights from a normal distribution with Recommended gain value?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
    'foward pass'
    def forward(self,X):
         
        a = X.float()


        for i in range(len(layers)-2):

            z = self.linears[i](a)

            a = self.activation(z)

        out = self.linears[-1](a)


        return out

In [8]:
# the physics-guided neural network
class PINN():
    def __init__(self, X, u, layers, lb, ub):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        # settings
        self.lambda_1 = torch.tensor([0.0], requires_grad=True).to(device)
        self.lambda_2 = torch.tensor([-6.0], requires_grad=True).to(device)
        self.k = torch.tensor([.1], requires_grad=True).to(device)  # 初始化一个波数（自己选择）  

        self.lambda_1 = torch.nn.Parameter(self.lambda_1)
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        self.k = torch.nn.Parameter(self.k)
        # deep neural networks
        self.u_nn = Solu_NN(layers).to(device)
        self.u_nn.register_parameter('lambda_1', self.lambda_1)
        self.u_nn.register_parameter('lambda_2', self.lambda_2)
        self.u_nn.register_parameter('k', self.k)
         # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.u_nn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        #记录损失
        self.loss_pde = []
        self.loss_data = []
        self.loss_total = []
        self.iter_list = []
        self.iter = 0
        
    def net_u(self, x, t):  
        u = self.u_nn(torch.cat([x, t], dim=1))
        return u
    
    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
        lambda_1 = self.lambda_1        
        lambda_2 = torch.exp(self.lambda_2)
        #lambda_2 = self.lambda_2
        k =self.k
        #k =torch.exp(self.k)
        u = self.net_u(x, t)
        
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_tt = torch.autograd.grad(u_t, t, grad_outputs=torch.ones_like(u_t), retain_graph=True, create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]

        #f = u_t + lambda_1 * u * u_x - lambda_2 * u_xx
        #q = (-(a1*np.pi)**2-(a2*np.pi)**2+k**2)*np.sin(a1*np.pi*self.x)*np.sin(a2*np.pi*self.t)
        f = u_tt + u_xx +k**2*u

        loss_g_x = torch.autograd.grad(f, x, grad_outputs=torch.ones_like(f), retain_graph=True, create_graph=True)[0]
        loss_g_t = torch.autograd.grad(f, t, grad_outputs=torch.ones_like(f), retain_graph=True, create_graph=True)[0]
        return f,loss_g_x,loss_g_t
    
    def loss_func(self):
        u_pred = self.net_u(self.x, self.t)
        f_pred,loss_g_x,loss_g_t = self.net_f(self.x, self.t)
        loss_data = torch.mean((self.u - u_pred)**2)
        loss_pde = torch.mean(f_pred**2 )
        loss_gradient =torch.mean(loss_g_x**2)+torch.mean(loss_g_t**2)
        loss = loss_data+loss_pde#+loss_gradient
        self.optimizer.zero_grad()
        loss.backward()
        
        self.iter += 1
        if self.iter % 100 == 0:
            (
            print('Loss: %e, loss_data: %e, loss_pde: %e,  loss_gra: %e, lambda0_1: %.5f, lambda0_2: %.5f, k: %.5f' )
                

                    loss.item(), 
                    loss_data.item(), 
                    loss_pde.item(), 
                    loss_gradient.item(),
                    self.lambda_1.item(), 
                    torch.exp(self.lambda_2.detach()).item(),
                    #self.lambda_2.item(),
                    self.k.item()
                    #torch.exp(self.k.detach()).item(),
                    #形成曲线

            
            )
             
        return loss
    
    def train(self, nIter):
        self.u_nn.train()
        # Backward and optimize

        self.loss_pde = []
        self.loss_data = []
        self.loss_total = []
        self.iter_list = []

        self.optimizer.step(self.loss_func)
    
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.u_nn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

SyntaxError: invalid syntax (4064861893.py, line 98)

## Training on Non-noisy Data

In [4]:
%%time

noise = 0.0            
N_u = 5000
# create training set
np.random.seed(2235)
# idx = np.random.choice(X_star.shape[0], N_u, replace=False)
# X_u_train = X_star[idx,:]
# u_train = u_star[idx,:]

X_u_train = X_star
u_train = u_star

# training
model = PINN(X_u_train, u_train, layers, lb, ub)
loss_data_init = torch.mean((model.u - model.net_u(model.x,model.t)) ** 2)
loss_pde_init = torch.mean(model.net_f(model.x,model.t)[0] ** 2)
loss_gra_x_init = torch.mean(model.net_f(model.x,model.t)[1] ** 2)
loss_gra_y_init = torch.mean(model.net_f(model.x,model.t)[2] ** 2)
print('loss_data_init: %e'%loss_data_init.item())
print('loss_pde_init: %e'%loss_pde_init.item())
print('loss_gra_x_init: %e'%loss_gra_x_init.item())
print('loss_gra_y_init: %e'%loss_gra_y_init.item())

NameError: name 'PINN' is not defined

In [6]:
model.train(20000)

Loss: 2.565561e-02, loss_data: 2.466564e-02, loss_pde: 9.899720e-04,  loss_gra: 1.593132e-02, lambda0_1: 0.00000, lambda0_2: 0.00248, k: 1.55904
Loss: 2.420829e-02, loss_data: 2.268021e-02, loss_pde: 1.528083e-03,  loss_gra: 2.243961e-02, lambda0_1: 0.00000, lambda0_2: 0.00248, k: 1.74923
Loss: 2.386292e-02, loss_data: 2.243884e-02, loss_pde: 1.424073e-03,  loss_gra: 2.144759e-02, lambda0_1: 0.00000, lambda0_2: 0.00248, k: 1.77828
Loss: 2.353103e-02, loss_data: 2.201733e-02, loss_pde: 1.513693e-03,  loss_gra: 2.299281e-02, lambda0_1: 0.00000, lambda0_2: 0.00248, k: 1.81977
Loss: 2.325147e-02, loss_data: 2.170023e-02, loss_pde: 1.551241e-03,  loss_gra: 2.208852e-02, lambda0_1: 0.00000, lambda0_2: 0.00248, k: 1.84004
Loss: 2.286558e-02, loss_data: 2.144413e-02, loss_pde: 1.421450e-03,  loss_gra: 2.364868e-02, lambda0_1: 0.00000, lambda0_2: 0.00248, k: 1.91659
Loss: 2.213837e-02, loss_data: 2.106290e-02, loss_pde: 1.075466e-03,  loss_gra: 1.791439e-02, lambda0_1: 0.00000, lambda0_2: 0.002

In [12]:
torch.exp(model.k)

tensor([0.1698], device='cuda:0', grad_fn=<ExpBackward0>)

In [46]:
# evaluations
u_pred, f_pred = model.predict(X_star)

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)

U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')

lambda_1_value = model.lambda_1.detach().cpu().numpy()
lambda_2_value = model.lambda_2.detach().cpu().numpy()
#lambda_2_value = np.exp(lambda_2_value)

error_lambda_1 = np.abs(lambda_1_value - 1.0) * 100
error_lambda_2 = np.abs(lambda_2_value - nu) / nu * 100

print('Error u: %e' % (error_u))    
print('Error l1: %.5f%%' % (error_lambda_1))                             
print('Error l2: %.5f%%' % (error_lambda_2))  

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.67 GiB already allocated; 0 bytes free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
noise = 0.01    

# create training set
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])

# training
model = PINN(X_u_train, u_train, layers, lb, ub)
model.train(0)            

In [ ]:
# evaluations
u_pred, f_pred = model.predict(X_star)

lambda_1_value_noisy = model.lambda_1.detach().cpu().numpy()
lambda_2_value_noisy = model.lambda_2.detach().cpu().numpy()
lambda_2_value_noisy = np.exp(lambda_2_value_noisy)

error_lambda_1_noisy = np.abs(lambda_1_value_noisy - 1.0) * 100
error_lambda_2_noisy = np.abs(lambda_2_value_noisy - nu) / nu * 100

print('Error u: %e' % (error_u))    
print('Error l1: %.5f%%' % (error_lambda_1_noisy))                             
print('Error l2: %.5f%%' % (error_lambda_2_noisy))  

## Plots

In [ ]:

""" The aesthetic setting has changed. """

####### Row 0: u(t,x) ##################    

fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot(111)

h = ax.imshow(U_pred.T, interpolation='nearest', cmap='rainbow', 
              extent=[t.min(), t.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.10)
cbar = fig.colorbar(h, cax=cax)
cbar.ax.tick_params(labelsize=15) 

ax.plot(
    X_u_train[:,1], 
    X_u_train[:,0], 
    'kx', label = 'Data (%d points)' % (u_train.shape[0]), 
    markersize = 4,  # marker size doubled
    clip_on = False,
    alpha=.5
)

line = np.linspace(x.min(), x.max(), 2)[:,None]
ax.plot(t[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(t[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(t[75]*np.ones((2,1)), line, 'w-', linewidth = 1)

ax.set_xlabel('$t$', size=20)
ax.set_ylabel('$x$', size=20)
ax.legend(
    loc='upper center', 
    bbox_to_anchor=(0.9, -0.05), 
    ncol=5, 
    frameon=False, 
    prop={'size': 15}
)
ax.set_title('$u(t,x)$', fontsize = 20) # font size doubled
ax.tick_params(labelsize=15)

plt.show()

In [ ]:
####### Row 1: u(t,x) slices ################## 

""" The aesthetic setting has changed. """

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111)

gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1.0/3.0-0.1, bottom=1.0-2.0/3.0, left=0.1, right=0.9, wspace=0.5)

ax = plt.subplot(gs1[0, 0])
ax.plot(x,Exact[25,:], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,U_pred[25,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$u(t,x)$')    
ax.set_title('$t = 0.25$', fontsize = 15)
ax.axis('square')
ax.set_xlim([-1.1,1.1])
ax.set_ylim([-1.1,1.1])

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(15)

ax = plt.subplot(gs1[0, 1])
ax.plot(x,Exact[50,:], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,U_pred[50,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$u(t,x)$')
ax.axis('square')
ax.set_xlim([-1.1,1.1])
ax.set_ylim([-1.1,1.1])
ax.set_title('$t = 0.50$', fontsize = 15)
ax.legend(
    loc='upper center', 
    bbox_to_anchor=(0.5, -0.15), 
    ncol=5, 
    frameon=False, 
    prop={'size': 15}
)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(15)

ax = plt.subplot(gs1[0, 2])
ax.plot(x,Exact[75,:], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,U_pred[75,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$u(t,x)$')
ax.axis('square')
ax.set_xlim([-1.1,1.1])
ax.set_ylim([-1.1,1.1])    
ax.set_title('$t = 0.75$', fontsize = 15)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(15)

plt.show()